#Cleaning script for incoming DM form. Process to include:
# a) First clean and renaming of columns from raw form
# b) filter by watershed and ACO flight number
# c) Adding of coordinate information and importing of GNSS data 

In [2]:
#import libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [72]:
#flight/phase number? (AUTOMATE?)
flt_no = '1'

#open the raw csv, select specific columns to keep, drop additional columns
path = r"G:\ACO\2024"
# data = pd.read_csv(path + r"\1_scripts\working\p1\CRU_24_P01.csv", encoding='mac-roman')

df = pd.read_csv("CRU_24_P01.csv")

In [73]:
# define columns to keep and new column names
cols2keep = ["Survey_Start_Time",
          "username", 
          "Study_Area", 
          "Other_Study_Area", 
          "User_s_", 
          "Plot_ID", 
          "Tube_Name", 
          "Pre_Survey_Notes", 
          "GNSS_Used_", 
          "GNSS_Setup", 
          "Other_GNSS_Setup",
          "GNSS_Status",
          "GNSS_Height_Rover_to_Snow__cm_", 
          "Tare_Weight__g_",
          "Type_of_Plot", 
          "Point_Observation.Cardinal_Direction", 
          "Point_Observation.Distance_From_Centre__m_",
          "Point_Observation.Custom_Distance_From_Centre__m_",
          "Point_Observation.Plot_Features", 
          "Point_Observation.Sample_Type", 
          "Point_Observation.Depth__cm_", 
          "Point_Observation.Depth__cm_:timestamp", 
          "Point_Observation.Depth_Final__cm_",
          "Point_Observation.Core_Length__cm_", 
          "Point_Observation.Plug__cm_", 
          "Point_Observation.Mass___Tube__g_",
          "Point_Observation.Multi_Part_Core_",
          "Point_Observation.Multi_Part_Core_Section_Number", 
          "Point_Observation.Additional_Measurements.Multi_Part_Core_Section_Number_copy",
          "Point_Observation.Additional_Measurements.New_Depth__cm_", 
          "Point_Observation.Additional_Measurements.New_Depth__cm_:timestamp", 
          "Point_Observation.Additional_Measurements.Core_Section_Length__cm_",
          "Point_Observation.Additional_Measurements.Plug_", 
          "Point_Observation.Additional_Measurements.Mass___Tube", 
          "Point_Observation.Core_Length_Final__cm_",
          "Point_Observation.Core_Features", 
          "Point_Observation.Depth_of_Saturation", 
          "Point_Observation.Mass_Final__g_", 
          "Point_Observation.Depth_Max",
          "Point_Observation.Retrieval____", 
          "Point_Observation.SWE__cm_", 
          "Point_Observation.Density", 
          "Point_Observation.Sample_Rating", 
          "Point_Observation.Point_Observation_Notes", 
          "Snow_Pit_Measurement.Distance_from_centre_of_plot__m_", 
          "Snow_Pit_Measurement.Depth_above_ground__cm_", 
          "Snow_Pit_Measurement.Temperature___C_",
          "Snow_Pit_Measurement.Density_Notes", 
          "Survey_End_TIme"]

new_colnames = ["plot_datetime",
                    "user_name",
                    "study_area",
                    "other_study_area",
                    "users",
                    "plot_id",
                    "tube_name",
                    "pre_survey_notes",
                    "gnss_unit",
                    "gnss_setup",
                    "other_gnss_setup",
                    "gnss_status",
                    "rover_height",
                    "tare_weight_g",
                    "plot_type",
                    "cardinal",
                    "distance_m",
                    "custom_distance",
                    "plot_features",
                    "sample_type",
                    "depth_cm",
                    "depth_timestamp",
                    "depth_final_cm",
                    "core_length_cm",
                    "plug_cm",
                    "mass_tube_g",
                    "multi_core",
                    "multi_part_core_num",
                    "multi_core_num_copy",
                    "depth_cm_new",
                    "depth_cm_timestamp_new",
                    "core_section_length_cm",
                    "plug_cm_copy",
                    "mass_tube_g_new",
                    "core_length_final",
                    "core_features",
                    "depth_of_saturation",
                    "mass_final_g",
                    "depth_max",
                    "retrieval",
                    "swe_cm",
                    "density",
                    "sample_rating",
                    "notes",
                    "snow_pit_distance_from_centre",
                    "snow_pit_depth_above_ground",
                    "snow_pit_temperature",
                    "snow_pit_density",
                    "survey_end"]

In [74]:
# extract and rename columns
df = df[cols2keep].set_axis(new_colnames, axis='columns')
#  add ACO flight no.
df.insert(0, 'aco_flight_number', str(flt_no))

#Take the cleaned form and filter out by watershed
#export summary statistics

In [75]:
# fill nan depths with 0
df[['depth_final_cm', 'depth_max']] = df[['depth_final_cm', 'depth_max']].fillna(value=0)
# calculate and add snow depth
df = df.insert(25, 'snow_depth', df.loc[:, 'depth_final_cm'] + df.loc[:, 'depth_max'])

In [32]:
#export the clean raw device magic form to a csv
df.to_csv("processing_data\DM_trip" + str(flt_no) + "_clean_cru.csv", index = False)

In [33]:
#filter by watershed: "Russell Creek", "Cruickshank", "Englishman", or "Metro Vancouver"
cru_df = df[df['study_area'] == 'Cruickshank']
eng_df = df[df['study_area'] == 'Englishman']
rus_df = df[df['study_area'] == 'Russell Creek']
mv_df = df[df['study_area'] == 'Metro Vancouver']

In [35]:
#group by "plot_id", export csv of summary stats for snow. 
#Stats that are nice to include are: 
#snow depth:count, average, st dev; density: count, avg, st dev, & SWE : count, avg, st.dev

###CRUICKSHANK###

cru_sum = cru_df.groupby(['aco_flight_number',
    'plot_id']).agg({
    "snow_depth": ["mean", "median", "std", "count"],
    "density": ["mean", "median", "std", "count"],
    "swe_cm": ["mean", "median", "std", "count"]
})

#check if directory exists
if not os.path.isdir(path + r"\Cruickshank\4_field_data\plots\working\P" + str(flt_no)):
    #if the folder path is not presnet, then create it
    os.makedirs(path + r"\Cruickshank\4_field_data\plots\working\P" + str(flt_no))
    
#Export the filtered summary stats to created folder
cru_sum.to_csv(path + "\Cruickshank\\4_field_data\plots\working\P" + str(flt_no) + "\DM_trip" + str(flt_no) +"_cru_sum.csv", index = True)

In [23]:
###ENGLISHMAN###

eng_sum = eng_df.groupby(['aco_flight_number',
    'plot_id']).agg({
    "snow_depth": ["mean", "median", "std", "count"],
    "density": ["mean", "median", "std", "count"],
    "swe_cm": ["mean", "median", "std", "count"]
})

#check if directory exists
if not os.path.isdir(path + r"\Englishman\4_field_data\plots\working\P" + str(flt_no)):
    #if the folder path is not presnet, then create it
    os.makedirs(path + r"\Englishman\4_field_data\plots\working\P" + str(flt_no))
    
#Export the filtered summary stats to created folder
eng_sum.to_csv(path + "\Englishman\\4_field_data\plots\working\P" + str(flt_no) + "\DM_trip" + str(flt_no) +"_eng_sum.csv", index = True)

In [17]:
###TSITIKA####
rus_sum = rus_df.groupby(['aco_flight_number',
    'plot_id']).agg({
    "snow_depth": ["mean", "median", "std", "count"],
    "density": ["mean", "median", "std", "count"],
    "swe_cm": ["mean", "median", "std", "count"]
})

#check if directory exists
if not os.path.isdir(path + r"\Tsitika\4_field_data\plots\working\P" + str(flt_no)):
    #if the folder path is not presnet, then create it
    os.makedirs(path + r"\Tsitika\4_field_data\plots\working\P" + str(flt_no))
    
#Export the filtered summary stats to created folder
rus_sum.to_csv(path + "\Tsitika\\4_field_data\plots\working\P" + str(flt_no) + "\DM_trip" + str(flt_no) +"_rus_sum.csv", index = True)